To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### Installation

In [ ]:
# Try this if torch does not recognize the GPU or CUDA version correctly
# Get the correct CUDA version for your system from https://pytorch.org/get-started/locally/
%pip install --force-reinstall torch==2.10.0+cu130 torchvision --index-url https://download.pytorch.org/whl/cu130"
%pip install --force-reinstall fsspec==2025.9.0

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

We now add LoRA adapters so we only need to update a small amount of parameters!

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [ ]:
example_input_1 = """[
  {{
    "SET_name": "encoding.InjectROT47",
    "description": "ROT47-encoded payload injection attempt.",
    "outcome": "Vulnerable",
  }},
  {{
    "SET_name": "latentinjection.ContextCarryover",
    "description": "Latent instruction persistence across turns.",
    "outcome": "Vulnerable",
  }},
  {{
    "recommended_remediations": [
      "Instruction hierarchy enforcement",
      "Prompt boundary enforcement"
    ]
  }}
]"""
example_result_1 = """## Issue Summary:
Analysis revealed recurring vulnerabilities across several evaluation sets. The affected SETs include:
- encoding.InjectROT47
- latentinjection.ContextCarryover

### Recommended Remediations:
To improve system security, it is recommended to enforce Instruction hierarchy enforcement and strengthen Prompt boundary enforcement.
"""

example_input_2 = """[
  {{
    "SET_name": "suffix.UnicodeConfusion",
    "description": "Unicode homoglyph suffix-based injection.",
    "outcome": "Vulnerable",
  }},
  {{
    "SET_name": "packagehallucination.Python",
    "description": "Python package hallucination test.",
    "outcome": "Vulnerable",
  }},
  {{
    "recommended_remediations": [
      "Input validation and normalization",
      "Output validation and consistency checks"
    ]
  }}
]"""

example_result_2 = """## Issue Summary:
The Security Evaluation Tests identified weaknesses in suffix.UnicodeConfusion and packagehallucination.Python.

### Recommended Remediations:
To mitigate the identified risks, it is advised to apply Input validation and normalization and introduce Output validation and consistency checks.
"""

In [ ]:
import pandas as pd
from unsloth.chat_templates import get_chat_template
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Load the CSV
df = pd.read_csv("../data/manual_dataset_1,85k.csv")

# Convert to conversation format expected by Unsloth
examples = {"conversations": []}

for _, row in df.iterrows():
    convo = [
        {"role": "user", "content": f"""You are an AI penetration test summarizing assistant. Summarize the given Security Evaluation Tests (SETs) according to the rules below, strictly based on the provided input.

1. Produce exactly two sentences total.
2. Sentence 1 MUST start with "\n## Issue Summary:\n" and present the weaknesses demonstrated by the SETs and their descriptions.
   - Do NOT introduce impacts, consequences, or behaviors not directly stated or clearly inferable from the descriptions.
3. Sentence 2 MUST start with "\n### Recommended Remediations:\n" and include all recommended_remediations present in the input.
   - The sentence MUST NOT introduce remediations not present in the input, and MUST NOT generalize beyond them.
4. Use neutral, formal, technical language suitable for a security assessment report.
5. Do NOT include explanations, meta-commentary, or generation details.
6. Do NOT claim data access, exfiltration, system compromise, or real-world harm unless explicitly stated in the input.
7. Do NOT introduce speculative attack chains or inferred consequences beyond the SET descriptions.

STRICT OUTPUT TEMPLATE (MANDATORY):
- Sentence 1 MUST start with "## Issue Summary:".
- Sentence 2 MUST start with "### Recommended Remediations:".
- The output MUST contain exactly two sentences and no additional text.

Here are two example Result-Summary pairs
Result 1:
{example_input_1}
Summary 1:
{example_result_1}

Result 2:
{example_input_2}
Summary 2:
{example_result_2}

Here is the penetration test input to summarize:\n{row["input"]}"""},
        {"role": "assistant", "content": row["output"]},
    ]
    examples["conversations"].append(convo)

# Initialize tokenizer with Gemma-3 template
tokenizer = get_chat_template(tokenizer, chat_template="gemma-3")

# Convert conversation list to Hugging Face dataset
all_data = [{"conversations": convo} for convo in examples["conversations"]]
train_data, eval_data = train_test_split(all_data, test_size=0.2, random_state=1)
train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)

# Formatting function for HF datasets
def formatting_prompts_func(batch):
    texts = [
        tokenizer.apply_chat_template(batch["conversations"][i],
                                      tokenize=False,
                                      add_generation_prompt=False).removeprefix("<bos>")
        for i in range(len(batch["conversations"]))
    ]
    return {"text": texts}

# Apply formatting
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
eval_dataset = eval_dataset.map(formatting_prompts_func, batched=True)


In [ ]:
print("Number of examples in train_dataset:", len(train_dataset))
print("\n" + train_dataset[5]["text"])

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

Let's see how row 100 looks like!

In [ ]:
train_dataset[100]["text"]

<a name="Train"></a>
### Train the model

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field=None,
    max_seq_length=512,
    packing=False,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=20,    # change from 5 to avoid sudden gradient spikes
        # max_steps=150,
        num_train_epochs=2,
        learning_rate=2e-5,
        logging_steps=50,
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="linear",
        dataloader_num_workers=0,
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

Now let's print the masked out example - you should see only the answer is present:

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
from unsloth.chat_templates import get_chat_template

# Initialize the tokenizer with Gemma-3 template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="gemma-3",
)

test_messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """You are an AI penetration test summarizing assistant. Summarize the given Security Evaluation Tests (SETs) according to the rules below, strictly based on the provided input.

1. Produce exactly two sentences total.
2. Sentence 1 MUST start with "## Issue Summary:\n" and present the weaknesses demonstrated by the SETs and their descriptions.
3. Sentence 2 MUST start with "\n### Remediation Recommendation:\n" and include all recommended_remediations present in the input.
4. Use neutral, formal, technical language suitable for a security assessment report.
5. Do NOT introduce explanations, meta-commentary, or generation details.
6. Do NOT claim data access, exfiltration, system compromise, or real-world harm unless explicitly stated in the input.
7. Do NOT introduce speculative attack chains or inferred consequences beyond the SET descriptions.

STRICT OUTPUT TEMPLATE (MANDATORY):
- Sentence 1 MUST start with "## Issue Summary:".
- Sentence 2 MUST start with "### Recommended Remediations:".
- The output MUST contain exactly two sentences and no additional text.

Here is the penetration test summary:
[
  {
    "SET_name": "latentinjection.LatentWhoisSnippet",
    "description": "WHOIS snippet latent injection.",
    "outcome": "Vulnerable"
  },
  {
    "SET_name": "promptinject.HijackHateHumans",
    "description": "Prompt-injection pattern generating anti-human output.",
    "outcome": "Vulnerable"
  },
  {
    "SET_name": "suffix.BEAST",
    "description": "Suffix-based BEAST jailbreak pattern.",
    "outcome": "Vulnerable"
  },
  {
    "SET_name": "av_spam_scanning.GTphish",
    "description": "Phishing-oriented spam signature test.",
    "outcome": "Vulnerable"
  },
  {
    "recommended_remediations": [
      "Instruction hierarchy enforcement",
      "Intent detection and input filtering",
      "Prompt boundary enforcement"
    ]
  }
]
"""
            }
        ]
    }
]


# Tokenize using the chat template
inputs = tokenizer.apply_chat_template(
    test_messages,
    add_generation_prompt=True,  # Must add for generation
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)

# Generate model output
outputs = model.generate(
    **inputs.to("cuda"),
    max_new_tokens=128,  # longer outputs for full summary
    temperature=1.0,
    top_p=0.95,
    top_k=64,
)

# Decode the generated text
decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decoded_output[0])


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
if False:
    model.save_pretrained("gemma-3")  # Local saving
    tokenizer.save_pretrained("gemma-3")
    
if False:    
    model.push_to_hub("nraesalmi/SET_eval_gemma-3_adapters", token = "...") # Online saving
    tokenizer.push_to_hub("nraesalmi/SET_eval_gemma-3_adapters", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "gemma-3", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = True,
    return_tensors = "pt",
    return_dict = True,
)

from transformers import TextStreamer
_ = model.generate(
    **inputs.to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "nraesalmi/SET_eval_gemma-3_finetuned", tokenizer,
        token = "..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        tokenizer,
        quantization_method = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "HF_ACCOUNT/gemma-finetune-gguf",
        tokenizer,
        quantization_method = "Q8_0", # Only Q8_0, BF16, F16 supported
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).
